In [1]:
from config import *
from conversion import *
import json
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval


In [2]:
#Evaluates the labels in target_source agains the ground truth values in labels_source
def runCOCO(labels_source, target_source, single_cat = True, run_name=""):
  gt = COCO(labels_source)

  for ann in gt.anns:
    #Merge all labels into one category
    if(single_cat):
      gt.anns[ann]['category_id'] = 1
    gt.anns[ann]['iscrowd'] = 0
  revDict = {}

  for k in gt.imgs.keys():
    img = gt.imgs[k]
    revDict[img['file_name']] = img['id']

  with open(target_source) as f:
    json_str = f.read()

  data = json.loads(json_str)

  data['categories'] = gt.cats

  new_anns = []
  for ann in data['annotations']:
    image_name = [x['file_name'] for x in data['images'] if x['id']==ann['image_id']][0]
    if(image_name not in revDict.keys()):
      continue
    if(not ann['segmentation']):
      continue
    ann['image_id'] = revDict[image_name]
    #Manually merging categories
    if(ann['category_id'] in [0, 1, 4]):
      ann['category_id'] = 1
    ann['iscrowd'] = 0
    new_anns.append(ann)


  res = gt.loadRes(new_anns)
  r = COCOeval(gt, res, iouType='segm')
  #Toggle to control whether categories are used in scoring. 
  # By default it's set to off, to avoid penalizing the baseline DEVA which needs to rely on SAM's classification
  r.params.useCats = not single_cat
  r.evaluate()
  r.accumulate()
  #Visualization for sanity checking.
  for i in range(0, 2):
    res.showAnns(res.loadAnns(res.getAnnIds(i)), draw_bbox=True, name=f"temp/{i}_res_{run_name}")
    gt.showAnns(gt.loadAnns(gt.getAnnIds(i)), draw_bbox=True, name=f"temp/{i}_gt")
  return r


In [3]:
#Sloppy test code. Set targetSource to your desired ground truth labels' path,
# and targets to an array of paths holding the data to be examined.

# testSource = "/home/rbe07/Documents/Google/data/sequences/Labels/A_hand.json"
# testSource = "/home/rbe07/Documents/Google/data/sequences/Labels/hand_labeled_corrected_labels.json"
# targetSource1 = '/home/rbe07/Documents/Google/datasets/CO_ckpt0.json'
# targetSource2 = '/home/rbe07/Documents/Google/datasets/CO_corrected_ckpt0.json'
# targetSource3 = '/home/rbe07/Documents/Google/datasets/CO_orig_ckpt0.json'
# targetSource1 = '/home/rbe07/Documents/Google/datasets/test_naive.json'
testSource = "/home/rbe07/Documents/Google/data/sequences/Labels/CO_hand.json"
targetSource1 = "/home/rbe07/Documents/Google/data/5_5/Labels/CardboardOcclusions_rev_labels_DEVA.json"
targetSource2 = "/home/rbe07/Documents/Google/data/5_5/Labels/CardboardOcclusionsCorrected_rev_labels_DEVA.json"
targetSource3 = "/home/rbe07/Downloads/output/pred.json"
# targetSource3 = "/home/rbe07/Documents/Google/data/3_5/Labels/CardboardOcclusions_rev_labels_DEVA.json"
# targetSource4 = "/home/rbe07/Documents/Google/data/5_3/Labels/CardboardOcclusions_rev_labels_DEVA.json"


# targets = [targetSource2, targetSource3, targetSource4, targetSource5]
# targets = [targetSource2, targetSource3, targetSource4, targetSource5]
targets = [targetSource1, targetSource2, targetSource3]


In [ ]:

rets = []

for i, ts in enumerate(targets):
  ret = runCOCO(testSource, ts, single_cat=True, run_name=i)
  rets.append(ret)

In [ ]:
for r, name in zip(rets, targets):
  print(f"------{name}------")
  r.summarize()
  print("------------")